In [ ]:
"""
This notebook shows how the llama-3 model returns the information about Amazon products without any training dataset.
Please, use this notebook on Google Colab. 
"""

In [ ]:
"""
Installing the dependencies.
"""

!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install xformers==0.0.27 "trl<0.9.0" peft accelerate bitsandbytes
!pip install transformers datasets

In [ ]:
"""
Define some model constants.
"""

import json
import torch

from datasets import load_dataset
from transformers import TrainingArguments
from trl import SFTTrainer
from unsloth import FastLanguageModel, is_bfloat16_supported

max_seq_length = 2048
dtype = None
load_in_4bit = True
fourbit_models = [
    "unsloth/mistral-7b-v0.3-bnb-4bit",
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/llama-3-8b-bnb-4bit",
    "unsloth/llama-3-8b-Instruct-bnb-4bit",
    "unsloth/llama-3-70b-bnb-4bit",
    "unsloth/Phi-3-mini-4k-instruct",
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/mistral-7b-bnb-4bit",
    "unsloth/gemma-7b-bnb-4bit",
]

In [ ]:
"""
Loading the model.
"""

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

model = FastLanguageModel.for_inference(model)

In [ ]:
from transformers import TextStreamer

prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

inputs = tokenizer([
    prompt.format(
        "You are a Amazon assistant and must to provide additional information about any specific product the user requested.", 
        "Tell me about Ballet Dress-Up Fairy Tutu/ Which are the sizes avaiable ?",  
        "",
    )
], return_tensors = "pt").to("cuda")


text_streamer = TextStreamer(tokenizer)
outputs = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)